In [56]:
import pandas as pd

df = pd.read_csv("/Users/Jiwon/Documents/GitHub/advanced_project/Airbnb_project_15/add_poi.csv")

In [57]:
df

,source,name,description,neighborhood_overview,host_id,host_name,host_since,host_location,host_about,host_response_time,...,room_type_Shared room,structure_encoded,log_price,room_new_type,host_verifications_list,host_verifications_count,transport_count,infrastructure_count,tourism_count,poi_pca1
0,city scrape,Lg Rm in Historic Prospect Heights,Cozy space share in the heart of a great neigh...,Full of tree-lined streets and beautiful brown...,62165,Michael,2009,"New York, NY",I’m an urban planner working for an internatio...,NaN,...,False,23,5.303305,mid,"['email', 'phone', 'work_email']",3,7,77,1,-37.863477
1,city scrape,"1 Bedroom & your own Bathroom, Elevator Apartment",Private bedroom with your own bathroom in a 2 ...,"Manhattan, SE corner of 2nd Ave/ E. 110th street",157798,Irene,2010,"New York, NY",NaN,NaN,...,False,8,4.418841,upper-mid,"['email', 'phone']",2,3,79,4,-35.914362
2,city scrape,Spectacular West Harlem Garden Apt,This is a very large and unique space. An inc...,West Harlem is now packed with great restauran...,166532,Matthew,2010,"New York, NY",I have been a New Yorker for a long time\n and...,within an hour,...,False,14,4.941642,low-mid,"['email', 'phone']",2,4,34,2,-80.892434
3,city scrape,“Work-from-home” from OUR home.,*Monthly Discount will automatically apply <br...,THE NEIGHBORHOOD:<br />Our apartment is locate...,168525,Gustavo,2010,"New York, NY",I am a music producer. And my wife is a hair s...,within an hour,...,False,23,4.875197,mid,"['email', 'phone']",2,7,117,1,2.094658
4,city scrape,1 br in a 2 br apt (Midtown West),NaN,NaN,169927,Hubert,2010,"Saint-Aubin-sur-Scie, France","Facebook Likes:\r\nNew York French Geek, David...",NaN,...,False,23,4.941642,mid,"['email', 'phone']",2,15,335,3,220.203233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20555,city scrape,Beautiful 1-Bed Apt in Harlem!,Welcome to your temporary home in the vibrant ...,NaN,40019013,Cecilia,2015,"New York, NY",NaN,NaN,...,False,23,4.290459,mid,"['email', 'phone']",2,4,58,4,-56.857764
20556,city scrape,Private Room w/ Ensuite Bath H,Stylish Private Rooms w/ En-Suite Baths in Bro...,NaN,483056418,Kristina,2022,"New York, NY",Are you tired of the tedious and time-consumin...,within an hour,...,False,23,4.077537,mid,"['email', 'phone']",2,4,27,1,-87.915003
20557,city scrape,2 Bedroom on East Side,Located in the Murray Hill area and occupies a...,NaN,30283594,Global Luxury Suites,2015,"Hawthorne, NJ",NaN,within an hour,...,False,23,5.703782,mid,"['email', 'phone', 'work_email']",3,8,276,6,161.112336
20558,city scrape,Stylish 2Bd near Bryant Park,Enjoy everything the city has to offer while l...,NaN,407304997,Boomerang,2021,"New York, NY","At Boomerang, we believe in the idea ""you get ...",within an hour,...,False,23,5.303305,mid,"['email', 'phone']",2,19,340,9,225.515844


In [58]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1) PCA용 함수 정의
def add_pca_component(df, cols, new_col_name):
    """
    df[cols]에 대해 표준화→PCA(n_components=1) 실행 후
    첫 번째 PC 스코어를 df[new_col_name]으로 붙임.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[cols].fillna(0))
    pca = PCA(n_components=1, random_state=42)
    pc1 = pca.fit_transform(X_scaled).ravel()
    df[new_col_name] = pc1
    print(f"  • {new_col_name}: Explained variance = {pca.explained_variance_ratio_[0]:.3f}")
    return df

# 2) 그룹별 PCA 적용
# 2.1 응답 관련
resp_cols = ['host_response_time_score', 'host_response_rate_score']
df = add_pca_component(df, resp_cols, 'host_response_pca')

# 2.2 프로필/위치/인증 관련
prof_cols = ['host_has_profile_pic', 'host_location_ny', 'host_verifications_count']
df = add_pca_component(df, prof_cols, 'host_profile_pca')

# 2.3 숙소 규모/욕실 관련
room_cols = ['accommodates', 'beds', 'bath_score_mul']
df = add_pca_component(df, room_cols, 'room_size_pca')

# 2.4 숙소 정보 관련
info_cols = ['neighborhood_overview_exists', 'description_length_group', 'host_about_length_group', 'name_length_group']
df = add_pca_component(df, info_cols, 'room_info_pca')

# 2.5 price 평점 관련
price_cols = ['log_price', 'review_scores_rating',  'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value', 'structure_encoded']
df = add_pca_component(df, price_cols, 'price_info_pca')

# 3) 필요하다면 원본 컬럼(drop)  
# df = df.drop(resp_cols + prof_cols + room_cols, axis=1)


  • host_response_pca: Explained variance = 0.858
  • host_profile_pca: Explained variance = 0.454
  • room_size_pca: Explained variance = 0.688
  • room_info_pca: Explained variance = 0.348
  • price_info_pca: Explained variance = 0.605


In [59]:
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

In [60]:
df['reviews_per_month']

0        0.05
1        0.58
2        1.36
3        1.54
4        0.25
         ... 
20555    0.00
20556    0.00
20557    0.00
20558    0.00
20559    0.00
Name: reviews_per_month, Length: 20560, dtype: float64

In [61]:
df['minimum_nights'].isnull().sum()

np.int64(0)

In [65]:
df['availability_365'].isnull().mean()

np.float64(0.0)

In [63]:
df['price'].isnull().sum()

np.int64(0)

In [77]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

# ——————————————————————————————————————————
# 0) DataFrame 준비(df)
# ——————————————————————————————————————————
# df = pd.read_csv("your_data.csv")

# ——————————————————————————————————————————
# 1) 파생 변수 계산
# ——————————————————————————————————————————
df['occupancy'] = (df['reviews_per_month'] * df['minimum_nights'] * 12) / df['availability_365']
df['bookings_per_year'] = df['reviews_per_month'] * (1/0.5) * 12   # review_rate=0.5 가정
df['estimated_monthly_revenue'] = (
    df['reviews_per_month'] * (1/0.5) * df['price'] * df['minimum_nights']
)

# 무한대(inf, -inf) → 0으로 변환
df.replace([np.inf, -np.inf], 0, inplace=True)

# (필요하다면) NaN도 0으로 채우기
df.fillna(0, inplace=True)

X = df_clean[FEATURES]
y = df_clean[target_col]

# ——————————————————————————————————————————
# 2) 공통 설정
# ——————————————————————————————————————————
FEATURES = [
    'host_is_superhost','availability_365','instant_bookable',
    'calculated_host_listings_count','amenities_cnt',
    'is_private','is_activate','town_encoded','poi_pca1',
    'host_response_pca','host_profile_pca','room_size_pca',
    'room_info_pca','price_info_pca'
]

# 하이퍼파라미터 그리드
rf_params = {
    "n_estimators":      [100, 200, 500],
    "max_depth":         [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf":  [1, 2, 4]
}
cat_params = {
    "iterations":    [200, 500, 1000],
    "depth":         [4, 6, 8, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "l2_leaf_reg":   [1, 3, 5, 7],
    "border_count": [32, 64, 128]
}

def train_and_eval(target_col):
    print(f"\n========== Target: {target_col} ==========")
    X = df[FEATURES].fillna(0)
    y = df[target_col]

    # 1) split
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # 2) RandomizedSearchCV
    rf = RandomForestRegressor(random_state=42)
    cat = CatBoostRegressor(random_state=42, silent=True)

    rf_search = RandomizedSearchCV(
        rf, rf_params, n_iter=20, cv=3,
        scoring="neg_mean_squared_error",
        random_state=42, n_jobs=-1
    )
    rf_search.fit(X_tr, y_tr)
    best_rf = rf_search.best_estimator_

    cat_search = RandomizedSearchCV(
        cat, cat_params, n_iter=20, cv=3,
        scoring="neg_mean_squared_error",
        random_state=42, n_jobs=-1
    )
    cat_search.fit(X_tr, y_tr)
    best_cat = cat_search.best_estimator_

    print("✔ Best RF params :", rf_search.best_params_)
    print("✔ Best Cat params:", cat_search.best_params_)

    # 3) Stacking 앙상블
    stack = StackingRegressor(
        estimators=[("rf", best_rf), ("cat", best_cat)],
        final_estimator=RidgeCV(),
        cv=3,
        n_jobs=-1
    )
    stack.fit(X_tr, y_tr)

    # 4) 평가
    def _eval(name, model):
        y_pred = model.predict(X_te)
        print(f"--- {name} ---")
        print(f"RMSE : {root_mean_squared_error(y_te, y_pred):.2f}")
        print(f"MAE  : {mean_absolute_error(y_te, y_pred):.2f}")
        print(f"MAPE : {mean_absolute_percentage_error(y_te, y_pred):.2f}")
        print(f"R2   : {r2_score(y_te, y_pred):.3f}\n")

    _eval("RandomForest" , best_rf)
    _eval("CatBoost"     , best_cat)
    _eval("Stacking(RidgeCV)", stack)

# ——————————————————————————————————————————
# 3) 각 타깃별 실행
# ——————————————————————————————————————————
for tgt in ['occupancy','bookings_per_year','estimated_monthly_revenue']:
    train_and_eval(tgt)



========== Target: occupancy ==========
✔ Best RF params : {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 10}
✔ Best Cat params: {'learning_rate': 0.01, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 10, 'border_count': 128}
--- RandomForest ---
RMSE : 4.11
MAE  : 0.92
MAPE : 150933857154884.62
R2   : 0.333

--- CatBoost ---
RMSE : 4.20
MAE  : 1.03
MAPE : 563762545650353.00
R2   : 0.303

--- Stacking(RidgeCV) ---
RMSE : 4.02
MAE  : 1.01
MAPE : 571132117247418.25
R2   : 0.361


========== Target: bookings_per_year ==========
✔ Best RF params : {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 20}
✔ Best Cat params: {'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 10, 'border_count': 64}
--- RandomForest ---
RMSE : 40.56
MAE  : 14.80
MAPE : 2538916450589013.50
R2   : 0.260

--- CatBoost ---
RMSE : 39.72
MAE  : 14.87
MAPE : 4608413928831447.00
R2   : 0.290

--- Stacking(RidgeCV) ---
RMSE : 39.29
MAE

In [66]:
for col in ['occupancy','bookings_per_year','estimated_monthly_revenue']:
    print(col, df[col].isna().sum())

occupancy 47
bookings_per_year 0
estimated_monthly_revenue 0


In [70]:
df[df['occupancy'].isnull()]

,source,name,description,neighborhood_overview,host_id,host_name,host_since,host_location,host_about,host_response_time,...,tourism_count,poi_pca1,host_response_pca,host_profile_pca,room_size_pca,room_info_pca,price_info_pca,occupancy,bookings_per_year,estimated_monthly_revenue
101,city scrape,LL3,A large bedroom sleeps 3 in a modern 2 bedroom...,My home sits on a tree lined street across fro...,295128,Carol Gloria,2010,"New York, NY",I'm a world traveler and a retired customer s...,NaN,...,1,-115.025219,0.589146,0.469107,-0.185310,0.135980,0.022537,NaN,0.0,0.0
923,city scrape,LL2,This bedroom sleeps 2 guests and you Share the...,I live in a culturally diverse residential nei...,295128,Carol Gloria,2010,"New York, NY",I'm a world traveler and a retired customer s...,NaN,...,0,-115.055114,0.589146,0.469107,-1.170504,0.135980,0.005843,NaN,0.0,0.0
3870,city scrape,"Blueground | Midtown East, doorman & gym",Show up and start living from day one in Midto...,This furnished apartment is located in Murray ...,107434423,Blueground,2016,"New York, NY","We’re Blueground, a global proptech company wi...",within an hour,...,9,190.379381,0.589146,1.626309,-0.624136,1.270729,0.038938,NaN,0.0,0.0
3880,city scrape,"Blueground | UWS, balcony, doorman, gym",Show up and start living from day one in Uptow...,This furnished property is on the Upper West S...,107434423,Blueground,2016,"New York, NY","We’re Blueground, a global proptech company wi...",within an hour,...,8,-8.753804,0.589146,1.626309,-0.624136,1.270729,0.046738,NaN,0.0,0.0
3912,city scrape,"Blueground | Lenox Hill, doorman, nr Central Park","Discover the best of Uptown, with this one bed...",This furnished property is situated in Lenox H...,107434423,Blueground,2016,"New York, NY","We’re Blueground, a global proptech company wi...",within an hour,...,1,52.007707,0.589146,1.626309,-0.624136,1.270729,0.043269,NaN,0.0,0.0
9064,city scrape,"Blueground | UWS, gym & doorman, nr Julliard",Feel at home wherever you choose to live with ...,This furnished property is on the Upper West S...,107434423,Blueground,2016,"New York, NY","We’re Blueground, a global proptech company wi...",within an hour,...,6,-58.795881,0.589146,1.626309,-0.624136,1.270729,0.039687,NaN,0.0,0.0
10566,city scrape,Luminous 3 Bedroom UWS Penthouse with Terrace,"A stylish, serene penthouse with a wrap-around...",NaN,4316334,Jan,2012,"New York, NY",I am a film and television director who travel...,NaN,...,8,-0.831415,0.589146,1.626309,3.704294,0.576809,0.118246,NaN,0.0,0.0
11581,city scrape,"Blueground | UES, elevator & gym, nr Central Park",Show up and start living from day one in Uptow...,This furnished property is located on the Uppe...,107434423,Blueground,2016,"New York, NY","We’re Blueground, a global proptech company wi...",within an hour,...,3,6.046404,0.589146,1.626309,-0.624136,1.270729,0.030213,NaN,0.0,0.0
12650,city scrape,Classic 2 Queen Beds Room at Hotel Edison New ...,Step into the vibrant energy of New York City ...,Times Square - 0.1 miles; <br />Broadway - 0.1...,501999278,RoomPicks,2023,NaN,NaN,within an hour,...,9,267.298791,0.589146,-0.851801,0.720526,0.903440,0.032494,NaN,0.0,0.0
12658,city scrape,2 x Classic 2 Queen Beds Room at Hotel Edison NYC,Step into the vibrant energy of New York City ...,Times Square - 0.1 miles; <br />Broadway - 0.1...,501999278,RoomPicks,2023,NaN,NaN,within an hour,...,9,267.298791,0.589146,-0.851801,4.502588,0.903440,0.055647,NaN,0.0,0.0


In [79]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# ——————————————————————————————————————————
# 0) 파생변수 & High/Low 레이블 생성
# ——————————————————————————————————————————
# 파생변수
df['occupancy'] = (df['reviews_per_month'] * df['minimum_nights'] * 12) / df['availability_365']
df['bookings_per_year'] = df['reviews_per_month'] * 2 * 12
df['estimated_monthly_revenue'] = (
    df['reviews_per_month'] * 2 * df['price'] * df['minimum_nights']
)
# 무한대(inf, -inf) → 0으로 변환
df.replace([np.inf, -np.inf], 0, inplace=True)

# (필요하다면) NaN도 0으로 채우기
df.fillna(0, inplace=True)


# 평균 기준 High/Low
for col in ['occupancy', 'bookings_per_year', 'estimated_monthly_revenue']:
    mean_val = df[col].mean()
    df[f'high_{col}'] = (df[col] >= mean_val).astype(int)
    print(f"{col} mean = {mean_val:.3f}")

# ——————————————————————————————————————————
# 1) 공통 설정
# ——————————————————————————————————————————
FEATURES = [
    'host_is_superhost','availability_365','instant_bookable',
    'calculated_host_listings_count','amenities_cnt',
    'is_private','is_activate','town_encoded','poi_pca1',
    'host_response_pca','host_profile_pca','room_size_pca',
    'room_info_pca','price_info_pca'
]

rf_params = {
    "n_estimators":      [100, 200, 500],
    "max_depth":         [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf":  [1, 2, 4]
}
cat_params = {
    "iterations":    [200, 500, 1000],
    "depth":         [4, 6, 8, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "l2_leaf_reg":   [1, 3, 5, 7],
}

def run_classification(target_col):
    print(f"\n===== Predicting {target_col} =====")
    # 2) 데이터 준비 (결측/inf 필터링)
    df_clean = df[FEATURES + [target_col]].replace([np.inf, -np.inf], np.nan).dropna()
    X = df_clean[FEATURES]
    y = df_clean[target_col]

    # 3) train/test split
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # 4) RandomizedSearchCV로 최적 파라미터 찾기
    rf = RandomForestClassifier(random_state=42)
    cat = CatBoostClassifier(random_state=42, silent=True)

    rf_search = RandomizedSearchCV(
        rf, rf_params, n_iter=20, cv=3,
        scoring="roc_auc", random_state=42, n_jobs=-1
    )
    rf_search.fit(X_tr, y_tr)
    best_rf = rf_search.best_estimator_

    cat_search = RandomizedSearchCV(
        cat, cat_params, n_iter=20, cv=3,
        scoring="roc_auc", random_state=42, n_jobs=-1
    )
    cat_search.fit(X_tr, y_tr)
    best_cat = cat_search.best_estimator_

    print("✔ Best RF params :", rf_search.best_params_)
    print("✔ Best Cat params:", cat_search.best_params_)

    # 5) StackingClassifier
    stack = StackingClassifier(
        estimators=[("rf", best_rf), ("cat", best_cat)],
        final_estimator=LogisticRegression(max_iter=1000),
        cv=3,
        n_jobs=-1,
        passthrough=True
    )
    stack.fit(X_tr, y_tr)

    # 6) 평가
    y_pred = stack.predict(X_te)
    y_proba = stack.predict_proba(X_te)[:,1]

    print(f"Accuracy: {accuracy_score(y_te, y_pred):.3f}")
    print(f"ROC AUC : {roc_auc_score(y_te, y_proba):.3f}")
    print(classification_report(y_te, y_pred))


# ——————————————————————————————————————————
# 2) 실행
# ——————————————————————————————————————————
for tgt in ['high_occupancy', 'high_bookings_per_year', 'high_estimated_monthly_revenue']:
    run_classification(tgt)


occupancy mean = 1.197
bookings_per_year mean = 20.029
estimated_monthly_revenue mean = 3250.412

===== Predicting high_occupancy =====
✔ Best RF params : {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}
✔ Best Cat params: {'learning_rate': 0.01, 'l2_leaf_reg': 1, 'iterations': 1000, 'depth': 10}


/Users/Jiwon/Documents/GitHub/advanced_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.856
ROC AUC : 0.901
              precision    recall  f1-score   support

           0       0.88      0.95      0.91      3278
           1       0.70      0.50      0.59       834

    accuracy                           0.86      4112
   macro avg       0.79      0.72      0.75      4112
weighted avg       0.85      0.86      0.85      4112


===== Predicting high_bookings_per_year =====
✔ Best RF params : {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}
✔ Best Cat params: {'learning_rate': 0.01, 'l2_leaf_reg': 1, 'iterations': 1000, 'depth': 10}


/Users/Jiwon/Documents/GitHub/advanced_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.847
ROC AUC : 0.916
              precision    recall  f1-score   support

           0       0.89      0.90      0.89      2968
           1       0.74      0.70      0.72      1144

    accuracy                           0.85      4112
   macro avg       0.81      0.80      0.81      4112
weighted avg       0.85      0.85      0.85      4112


===== Predicting high_estimated_monthly_revenue =====
✔ Best RF params : {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}
✔ Best Cat params: {'learning_rate': 0.01, 'l2_leaf_reg': 1, 'iterations': 1000, 'depth': 10}


/Users/Jiwon/Documents/GitHub/advanced_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.830
ROC AUC : 0.886
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      3077
           1       0.70      0.57      0.63      1035

    accuracy                           0.83      4112
   macro avg       0.78      0.75      0.76      4112
weighted avg       0.82      0.83      0.82      4112



In [85]:
pip install imblearn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [87]:
pip install --upgrade scikit-learn

  Using cached scikit_learn-1.7.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.1-cp311-cp311-macosx_12_0_arm64.whl (8.7 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [90]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from scipy.stats import randint, uniform

# 0) 파생변수 & High/Low 레이블
df['occupancy'] = (df['reviews_per_month'] * df['minimum_nights'] * 12) / df['availability_365']
df['bookings_per_year'] = df['reviews_per_month'] * 2 * 12
df['estimated_monthly_revenue'] = (
    df['reviews_per_month'] * 2 * df['price'] * df['minimum_nights']
)
for col in ['occupancy','bookings_per_year','estimated_monthly_revenue']:
    df[f'high_{col}'] = (df[col] >= df[col].mean()).astype(int)

# 1) 특징 엔지니어링
df['price_size_interaction'] = df['price_info_pca'] * df['room_size_pca']

FEATURES = [
    'host_is_superhost','availability_365','instant_bookable',
    'calculated_host_listings_count','amenities_cnt',
    'is_private','is_activate','town_encoded','poi_pca1',
    'host_response_pca','host_profile_pca','room_size_pca',
    'room_info_pca','price_info_pca','price_size_interaction'
]

# 2) inf/NaN 처리
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# 3) 데이터 분할
TARGET = 'high_occupancy'  # 또는 'high_bookings_per_year', 'high_estimated_monthly_revenue'
X = df[FEATURES]
y = df[TARGET]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 4) 학습기 정의 (불균형 완화 via class_weight)
rf = RandomForestClassifier(class_weight='balanced', random_state=42)
gb = GradientBoostingClassifier(random_state=42)

# 5) 하이퍼파라미터 분포 정의
rf_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(5, 30),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}
gb_dist = {
    'n_estimators': randint(100, 1000),
    'learning_rate': uniform(0.01, 0.19),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.5, 0.5)
}

# 6) RandomizedSearchCV로 RF/GB 튜닝
rf_search = RandomizedSearchCV(
    rf, rf_dist, n_iter=20, cv=3,
    scoring='roc_auc', random_state=42, n_jobs=-1
)
gb_search = RandomizedSearchCV(
    gb, gb_dist, n_iter=20, cv=3,
    scoring='roc_auc', random_state=42, n_jobs=-1
)
rf_search.fit(X_train, y_train)
gb_search.fit(X_train, y_train)

best_rf = rf_search.best_estimator_
best_gb = gb_search.best_estimator_

print("▶ Best RF params:", rf_search.best_params_)
print("▶ Best GB params:", gb_search.best_params_)

# 7) StackingClassifier 구성
stack = StackingClassifier(
    estimators=[('rf', best_rf), ('gb', best_gb)],
    final_estimator=RandomForestClassifier(class_weight='balanced', random_state=42),
    passthrough=False,
    cv=3,
    n_jobs=-1
)
stack.fit(X_train, y_train)

# 8) 평가
y_pred = stack.predict(X_test)
y_proba = stack.predict_proba(X_test)[:,1]

print("\n=== Stacking Classification Report ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC :", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


▶ Best RF params: {'max_depth': 28, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 443}
▶ Best GB params: {'learning_rate': np.float64(0.01886647601058693), 'max_depth': 6, 'n_estimators': 882, 'subsample': np.float64(0.728034992108518)}

=== Stacking Classification Report ===
Accuracy: 0.9997568093385214
ROC AUC : 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4092
           1       1.00      0.95      0.97        20

    accuracy                           1.00      4112
   macro avg       1.00      0.97      0.99      4112
weighted avg       1.00      1.00      1.00      4112



In [91]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from scipy.stats import randint, uniform

# 0) 파생변수 & High/Low 레이블
df['occupancy'] = (df['reviews_per_month'] * df['minimum_nights'] * 12) / df['availability_365']
df['bookings_per_year'] = df['reviews_per_month'] * 2 * 12
df['estimated_monthly_revenue'] = (
    df['reviews_per_month'] * 2 * df['price'] * df['minimum_nights']
)
for col in ['occupancy','bookings_per_year','estimated_monthly_revenue']:
    df[f'high_{col}'] = (df[col] >= df[col].mean()).astype(int)

# 1) 특징 엔지니어링
df['price_size_interaction'] = df['price_info_pca'] * df['room_size_pca']

FEATURES = [
    'host_is_superhost','availability_365','instant_bookable',
    'calculated_host_listings_count','amenities_cnt',
    'is_private','is_activate','town_encoded','poi_pca1',
    'host_response_pca','host_profile_pca','room_size_pca',
    'room_info_pca','price_info_pca','price_size_interaction'
]

# 2) inf/NaN 처리
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# 3) 데이터 분할
TARGET = 'high_bookings_per_year'  # 또는 'high_bookings_per_year', 'high_estimated_monthly_revenue'
X = df[FEATURES]
y = df[TARGET]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 4) 학습기 정의 (불균형 완화 via class_weight)
rf = RandomForestClassifier(class_weight='balanced', random_state=42)
gb = GradientBoostingClassifier(random_state=42)

# 5) 하이퍼파라미터 분포 정의
rf_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(5, 30),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}
gb_dist = {
    'n_estimators': randint(100, 1000),
    'learning_rate': uniform(0.01, 0.19),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.5, 0.5)
}

# 6) RandomizedSearchCV로 RF/GB 튜닝
rf_search = RandomizedSearchCV(
    rf, rf_dist, n_iter=20, cv=3,
    scoring='roc_auc', random_state=42, n_jobs=-1
)
gb_search = RandomizedSearchCV(
    gb, gb_dist, n_iter=20, cv=3,
    scoring='roc_auc', random_state=42, n_jobs=-1
)
rf_search.fit(X_train, y_train)
gb_search.fit(X_train, y_train)

best_rf = rf_search.best_estimator_
best_gb = gb_search.best_estimator_

print("▶ Best RF params:", rf_search.best_params_)
print("▶ Best GB params:", gb_search.best_params_)

# 7) StackingClassifier 구성
stack = StackingClassifier(
    estimators=[('rf', best_rf), ('gb', best_gb)],
    final_estimator=RandomForestClassifier(class_weight='balanced', random_state=42),
    passthrough=False,
    cv=3,
    n_jobs=-1
)
stack.fit(X_train, y_train)

# 8) 평가
y_pred = stack.predict(X_test)
y_proba = stack.predict_proba(X_test)[:,1]

print("\n=== Stacking Classification Report ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC :", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


▶ Best RF params: {'max_depth': 28, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 443}
▶ Best GB params: {'learning_rate': np.float64(0.04239958350058539), 'max_depth': 9, 'n_estimators': 373, 'subsample': np.float64(0.9744427686266666)}

=== Stacking Classification Report ===
Accuracy: 0.813715953307393
ROC AUC : 0.8866519683147043
              precision    recall  f1-score   support

           0       0.86      0.89      0.87      2968
           1       0.68      0.62      0.65      1144

    accuracy                           0.81      4112
   macro avg       0.77      0.76      0.76      4112
weighted avg       0.81      0.81      0.81      4112

